In [14]:
import fitz  # PyMuPDF

def pdf_to_txt(pdf_path, txt_path):
    doc = fitz.open(pdf_path)
    with open(txt_path, 'w', encoding='utf-8') as f:
        for page in doc:
            text = page.get_text()
            f.write(text + '\n')

data_path = "/home/liusicen/methods/DynamicFusion/Datasets/datasets/Disprot504.pdf"
output_path = "/home/liusicen/methods/DynamicFusion/Datasets/datasets/Disprot504.txt"
# pdf_to_txt("example.pdf", "output.txt")
pdf_to_txt(data_path, output_path)


In [15]:
# 将pdf转成txt之后的文件转换为fasta文件
# data_path = "/home/liusicen/models/DeepDRP-my/datasets/SL329.txt"
count = 0
ids = []
seqs = []
labels = []
with open(output_path, 'r') as f:
    data = f.read()
# 按行分割数据
lines = data.strip().split('\n')

protein = []
current_protein = {}
for i, line in enumerate(lines):
    if line.startswith('>'):
        if current_protein:
            protein.append(current_protein)
            current_protein = {}
        current_protein['name'] = line.strip()
    elif line and line[0].isalpha(): # 蛋白质序列
        current_protein.setdefault('sequence', '') 
        current_protein['sequence'] += line.strip()
    elif line and line[0].isdigit(): # 标签
        current_protein.setdefault('label', '')
        current_protein['label'] += line.strip()
    
# 添加最后一个蛋白质
if current_protein:
    protein.append(current_protein)

for pro in protein:
    if len(pro['sequence']) != len(pro['label']):
        print(pro['name'])

import os
base_path = output_path.split(".txt")[0]  
new_path = f"{base_path}.fasta"   
print(new_path)   
print(len(protein))
protein_name = []
with open(new_path, 'w') as f:
    for pro in protein:
        if pro['name'] not in protein_name:
            protein_name.append(pro['name'])
            f.write(f"{pro['name']}\n")
            f.write(pro['sequence'] + "\n")
            f.write(pro['label'] + "\n")
    print(f"Saved {new_path}")
    print(f"protein nums {len(protein_name)}")
       

/home/liusicen/methods/DynamicFusion/Datasets/datasets/Disprot504.fasta
1008
Saved /home/liusicen/methods/DynamicFusion/Datasets/datasets/Disprot504.fasta
protein nums 504


In [16]:
protein_name = []
for pro in protein:
    protein_name.append(pro['name'])
len(protein_name), len(set(protein_name))

(1008, 504)

In [17]:
# 统计有序和无序残基的数量
import os
import numpy as np
class fastaExample(object):
    def __init__(self, id, name, seq, label):
        self.id = id
        self.name = name
        self.seq = seq
        self.label = label

def parseFasta_with_label(input_file):
    samples = []
    load_f = []
    with open(input_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line=line.strip('\n')
            load_f.append(line)
    for i in range(len(load_f)):
        if i % 3 == 0:
            id = load_f[i].strip('>').strip('\n\r')
            seq = load_f[i+1]
            label = load_f[i+2]
            samples.append(fastaExample(id, id, seq, label))
    return samples 
# path = "/home/liusicen/models/DeepDRP-my/datasets/MXD494.fasta"
# path ="/home/liusicen/models/DeepDRP-my/datasets/SL329.fasta"
new_path = "/home/liusicen/methods/DynamicFusion/Datasets/datasets/Disprot504.fasta"

prot_list = parseFasta_with_label(new_path)
len(prot_list)

zero_count = 0
one_count = 0
for prot in prot_list:
    for lab in prot.label:
        if lab == '0':
            zero_count += 1
        elif lab == '1':
            one_count += 1
print("ordered", zero_count, zero_count/(zero_count + one_count), "\n", "Disordered", one_count, one_count/(zero_count + one_count))

ordered 226992 0.7530104894408949 
 Disordered 74454 0.24698951055910512
